In [2]:
import os
import csv
from collections import defaultdict
import pandas as pd
from pandas_style import *
from constants import *

event_files = os.listdir("./results")

# Remove system file(s)
if ".DS_Store" in event_files:
    event_files.remove(".DS_Store")

# Sort events chronologically based on file creation times
event_times = []
for event_file in event_files:
    event_times.append(os.path.getmtime("./results/" + event_file))
event_files = [x for _,x in sorted(zip(event_times, event_files))]
    
# Get results for events
events = []
players = []
for event_file in event_files:
    with open("./results/" + event_file, mode='r') as f:
        csv_reader = csv.DictReader(f)
        event_results = {}
        for row in csv_reader:
            # Add user-score pair to event container
            event_results[row["USER"]] = row["WAID"]
            # Also make sure we have a complete list of all players
            if row["USER"] not in players:
                players.append(row["USER"])
        # Append this event to the global container
        events.append(event_results)

# Get score for each event
score = defaultdict(list)
for event in events:
    for player in players:
        # If player missed this event, add 0
        if player not in event.keys():
            score[player] += [0.0]
        else:
            score[player] += [float(event[player])]

### Output Table

In [3]:
# Finally, let's make the results in a nice table format
event_names = [e[:-4].replace("_", " ") for e in event_files]
# Make a Pandas dataframe
df = pd.DataFrame.from_dict(score, orient='index', columns=event_names)
# Add a sum & sort
df.loc[:,'Total'] = df.sum(axis=1)
df = df.sort_values(by='Total', ascending=False)

# Add stylings
df = df.style.\
    applymap(color_negative_red).\
    apply(highlight_event_winner).\
    applymap(bold, subset=pd.IndexSlice[:, 'Total'])

# Display in the notebook
df

,StarLadder Major Q,StarLadder Major,BLAST Moscow,ESL One New York,StarSeries i-League S8,BLAST Copenhagen,IEM Beijing,Dreamhack Open Atlanta,Total
RippSir,0.654545,0.345562,0.193971,0.384586,0.414751,0.405098,0.394688,0.426012,3.21921
KokyTheGod,0.652727,0.63787,-0.114286,0.382331,0.40613,0.447918,0.140625,0.390173,2.94349
Vrzman,0.379091,0.480473,0.133159,0.514286,0.485632,0.369754,0.045,0.460694,2.86809
Tademon,0.292727,0.246154,0.179292,0.446617,0.148467,0.500935,0.6675,0.361272,2.84296
Rakvis,0.500909,0.210651,0,0,0.269157,0.316058,0.638437,0,1.93521
impvlse,0.491818,0.190533,-0.366972,0.287594,0.302682,0.307901,0.328125,0.378035,1.91972
Polysmen,0.434545,0.223669,-0.123722,0,0.501916,0.0101954,-0.06,0.272254,1.25886
